# Chapter 6 : Data Loading, Storage, 

-  There are numerous tools in other libraries to help with reading and writing data in various formats.

- Input and output typically falls into a few main categories: 
  
  - reading text files and other more efficient on-disk formats, 
  - loading data from databases, 
  - and interacting with network sources like web APIs

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Reading and Writing Data in Text Format

- pandas features a number of functions for reading tabular data as a DataFrame object. 

- Because of how messy data in the real world can be, **read_csv()** have grown very complex in their options over time. 

- It’s normal to feel overwhelmed by the number of different parameters (read_csv has over 50 as of this writing). 

- The online pandas documentation has many examples about how each of them works, so if you’re struggling to read a particular file, there might be a similar enough example to help you find the right parameters

### Table 6.1: Parsing functions in pandas

![](./images/pandascsv.png)

In [90]:
#Let’s start with a small comma-separated (CSV) text file
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [92]:
#Since this is comma-delimited, we can use read_csv to read it into a DataFrame
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [93]:
!cat examples/ex1.csv 

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [95]:
!cat examples/ex1.csv | head -n 2

a,b,c,d,message
1,2,3,4,hello


In [98]:
#We could also have used read_table and specified the delimiter
pd.read_table('examples/ex1.csv', sep=',')


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> The difference between **read_csv()** and **read_table()** is almost nothing. In fact, the same function is called by the source: read_csv() delimiter is a comma character.

> A file will not always have a header row. 

In [99]:
#A file will not always have a header row. Consider this file
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yoursel

In [100]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [101]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- Suppose you wanted the message column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named 'message' using the index_col argument:

In [102]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


OR

In [103]:
pd.read_csv('examples/ex2.csv', names=names, index_col=4)

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Another Example

In [104]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In the event that you want to form a hierarchical index from multiple columns, pass a list of column numbers or names

In [105]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Another Example

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:

In [106]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

> In these cases, you can pass a regular expression as a delimiter for read_csv. This can be expressed by the regular expression \s+, so we have then:

In [107]:
result = pd.read_csv('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


> Because there was one fewer column name than the number of data rows, read_csv infers that the first column should be the DataFrame’s index in this special case.

- The parser functions have many additional arguments to help you handle the wide variety of exception file formats that occur (see below). 
  
- For example, you can skip the first, third, and fourth rows of a file with skiprows:

![](./images/parsercsv.png)

Another Example: Skipping rows

In [108]:
# !cat examples/ex4.csv
pd.read_csv('examples/ex4.csv')

,,,,# hey!
a,b,c,d,message
# just wanted to make things more difficult for you,NaN,NaN,NaN,NaN
# who reads CSV files with computers,anyway?,NaN,NaN,NaN
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [109]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> Handling Missing values

In [110]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [111]:
result = pd.read_csv('examples/ex5.csv')
result


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


The na_values option can take either a list or set of strings to consider missing values:

In [112]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Different NA sentinels can be specified for each column in a dict:

In [113]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

> When processing very large files or figuring out the right set of arguments to correctly process a large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file.

In [114]:
#For large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file

#Before we look at a large file, we make the pandas display settings more compact:

pd.options.display.max_rows = 10


In [115]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


> If you want to only read a small number of rows (avoiding reading the entire file), specify that with nrows

In [118]:
pd.read_csv('examples/ex6.csv', nrows=3)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G


In [123]:
file =  pd.read_csv('examples/ex6.csv')
file.head(7)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U


### Writing Data to Text Format

Data can also be exported to a delimited format. Let’s consider one of the CSV files read before:

In [124]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


> Using DataFrame’s to_csv method, we can write the data out to a comma-separated file

In [129]:
data.to_csv('examples/out.csv', columns = ["a"])
# data.to_csv('examples/datall.csv')


In [130]:
!cat examples/out.csv

,a
0,1
1,5
2,9


In [131]:
pd.read_csv('examples/out.csv')

,Unnamed: 0,a
0,0,1
1,1,5
2,2,9


In [132]:
pd.read_csv('examples/out.csv', index_col=0)

,a
0,1
1,5
2,9


> Series also has a to_csv method:

In [133]:
index = np.arange(10, 17)
index

array([10, 11, 12, 13, 14, 15, 16])

In [134]:
series = pd.Series(np.arange(7), index=index)
series

10    0
11    1
12    2
13    3
14    4
15    5
16    6
dtype: int64

In [135]:
series.to_csv('examples/series.csv')

In [ ]:
!cat examples/series.csv

In [136]:
pd.read_csv('examples/series.csv', index_col=0)

,0
10,0
11,1
12,2
13,3
14,4
15,5
16,6


### Working with Delimited Formats

- It's possible to load most forms of tabular data from disk using functions like pandas.read_csv. 

- In some cases, however, some manual processing may be necessary. 

- It’s not uncommon to receive a file with one or more malformed lines that trip up read_csv. 
 
- To illustrate the basic tools, consider the example below:

In [139]:
# !cat examples/ex7.csv

pd.read_csv('examples/ex7.csv', header=None)

,0,1,2
0,a,b,c
1,1,2,3
2,1,2,3


For any file with a single-character delimiter, you can use Python’s built-in csv module. To use it, pass any open file or file-like object to csv.reader:

In [141]:
import csv

f = open('examples/ex7.csv')

reader = csv.reader(f)

reader

Iterating through the reader like a file yields lists of values with any quote characters removed:

In [142]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


- From there, it’s up to you to do the wrangling necessary to put the data in the form that you need it. 

- Let's take this step by step change the file to dictionary

In [143]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

Then, we split the lines into the header line and the data lines:

In [144]:
header, values = lines[0], lines[1:]

Then we can create a dictionary of data columns using a dictionary comprehension and the expression zip(*values) (beware that this will use a lot of memory on large files), which transposes rows to columns:

In [145]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [146]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

### JSON Data



> JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. 

Here is an example:

In [152]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

- JSON is very nearly valid Python code with the exception of its null value null and some other nuances (such as disallowing trailing commas at the end of lists). 

- The basic types are objects (dicts), arrays (lists), strings, numbers, booleans, and nulls. 

- All of the keys in an object must be strings. 

- There are several Python libraries for reading and writing JSON data. 
  
- We will use built into the Python standard library. 

To convert a JSON string to Python form, use json.loads in json build-in Python library

In [154]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

> json.dumps, on the other hand, converts a Python object back to JSON

In [163]:
asjson = json.dumps(result)
asjson


'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

> How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. 

Conveniently, you can pass a list of dicts (which were previously JSON objects) to the DataFrame constructor and select a sub‐ set of the data fields:

In [164]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


> The pandas.read_json can automatically convert JSON datasets in specific arrange‐ ments into a Series or DataFrame. For example:

In [165]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


The default options for pandas.read_json assume that each object in the JSON array is a row in the table

In [166]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


> If you need to export data from pandas to JSON, one way is to use the to_json meth‐ ods on Series and DataFrame

In [159]:
print(data.to_json())
# print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


### XML and HTML: Web Scraping

conda install lxml
pip install beautifulsoup4 html5lib

In [160]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [161]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### Parsing XML with lxml.objectify

<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>

In [162]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/mta_perf/Performance_MNR.xml'

In [ ]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = pd.DataFrame(data)
perf.head()

In [ ]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [ ]:
root
root.get('href')
root.text

## Binary Data Formats : Pickle

- What is Pickle exactly? In Python, you can use the pickle module to serialize objects and save them to a file. You can then deserialize the serialized file to load them back when needed.

- Pickle has one major advantage over other formats — you can use it to store any Python object. 

- One of the most widely used functionalities is saving machine learning models after the training is complete. 

- That way, you don’t have to retrain the model every time you run the script.

> Pickle can be use to store Numpy arrays. It’s a no-brainer solution for setting checkpoints of some sort in your code.

- But 
  
  -  Pickle is Python-specific
  
  -  pickled files are Python-version-specific - You might encounter issues when saving files in one Python version and reading them in the other. Try to work in identical Python versions, if possible.
  
  -  Pickling doesn’t compress data — Pickling an object won’t compress it. Sure, the file will be smaller when compared to CSV, but you can compress the data manually for maximum efficiency.


- pandas objects all have a to_pickle method that writes the data to disk in pickle format:

In [167]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [168]:
frame.to_pickle('examples/frame_pickle')

You can read any "pickled" object stored in a file by using the built-in pickle directly, or even more conveniently using pandas.read_pickle:

In [169]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [ ]:
!rm examples/frame_pickle

> Caution : Pickle is only recommended as a short-term storage format. The problem is that it is hard to guarantee that the format will be stable over time; an object pickled today may not unpickle with a later version of a library. pandas has tried to maintain backward compatibility when possible, but at some point in the future it may be necessary to “break” the pickle format.


### Using HDF5 Format

> pandas has built-in support for several other open source binary data formats, such as HDF5, ORC, and Parquet. 

In [172]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

In [ ]:
store['obj1']

In [ ]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

In [ ]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

In [ ]:
os.remove('mydata.h5')

### Reading Microsoft Excel Files

To use ExcelFile, create an instance by passing a path to an xls or xlsx file:

In [173]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')
xlsx

Data stored in a sheet can then be read into DataFrame with parse

In [177]:
pd.read_excel(xlsx, 'Sheet1', index_col=0) 

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


If you are reading multiple sheets in a file, then it is faster to create the ExcelFile, but you can also simply pass the filename to pandas.read_excel

In [ ]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

To write pandas data to Excel format, you must first create an ExcelWriter, then write data to it using pandas objects’ to_excel method

In [ ]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

You can also pass a file path to to_excel and avoid the ExcelWriter:

In [ ]:
frame.to_excel('examples/ex2.xlsx')

In [ ]:
!rm examples/ex2.xlsx

## Interacting with Web APIs

Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one easy-to-use method  is the requests package.

Example: To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP request using the add-on requests library

In [ ]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

The Response object’s json method will return a dictionary containing JSON parsed into native Python objects:

In [ ]:
data = resp.json()
# data[0]['title']

Each element in data is a dictionary containing all of the data found on a GitHub issue page (except for the comments). We can pass data directly to DataFrame and extract fields of interest:

In [ ]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

## Interacting with Databases

In [ ]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

In [ ]:
!rm mydata.sqlite

## Conclusion